# Knights Move 6

In [1]:
pip install networkx

     |████████████████████████████████| 1.7 MB 7.1 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import networkx as nx

Pick distinct positive integers A, B, and C, and place them in the grid above. Your goal is to create two corner-to-corner trips — one from a1 to f6, and the other from a6 to f1 — both of which score exactly 2024 points.

A “trip” consists of knight’s moves. Squares may not be revisited within a trip.

The “score” for a trip is calculated as follows:

Start with A points.
Every time you make a move:

* if your move is between two different integers, multiply your score by the value you are moving to;

* otherwise, increment your score by the value you are moving to.

Can you find positive integers A, B, and C, as well as a pair of trips, that satisfy the criteria above? How low can you get A + B + C?

Please format your entry by concatenating your values for A, B, and C, followed by your a1-to-f6 tour, followed by your a6-to-f1 tour. For example, “1,2,253,a1,b3,c5,d3,f4,d5,f6,a6,c5,a4,b2,c4,d2,f1” would be a properly formatted entry.

To qualify for the leaderboard your value for A + B + C must be less than 50.

How low can you get A + B + C?

* BOARD: 1,2,253,
* TRIP 1: a1,b3,c5,d3,f4,d5,f6,
* TRIP 2: a6,c5,a4,b2,c4,d2,f1

2024 have prime factors of 1, 2, 11, 23, 

# Constraints

1. Squares are visited once at most within a trip
2. Start with A points
3. Every time you make a move:

* if your move is between two different integers, multiply your score by the value you are moving to;

* otherwise, increment your score by the value you are moving to.

4. A + B + C < 50
5. 1,2,253 works

# Thoughts

Creating a path to 2024 using any 3 digits?

Is that possible?

A grid with Knight's move creates a graph

Branch and Bound - Google OR Tools - Routing?

https://en.wikipedia.org/wiki/Subset_sum_problem

Simulated Annealing where energy starts at 2024
with conditions that it begins and terminates at given locations?

a1 -> b3 -> c5 -> d3 -> f4 -> d5 -> f6 
1 -> (1+1) -> 
2 * (1+1) -> 2 + (2 * (1+1)) -> 
253 * (2 + (2 * (1+1)) -> 253 + (253 * (2 + (2 * (1+1))) -> 
253 + 253 + (253 * (2 + (2 * (1+1)))
1 -> 2 ->

# Create Network

In [3]:
'''
for a 6x6 matrix where the columns are labeled as A, B, C, D, E, F and the rows are labeled as 1,2,3,4,5,6, 

create a graph of reachable nodes based on the movements of a knight in chess. A1 is the bottom left corner of the matrix

please represent the graph in python
'''
from collections import defaultdict

# Define the knight's moves
knight_moves = [
    (2, 1), (2, -1), (-2, 1), (-2, -1),
    (1, 2), (1, -2), (-1, 2), (-1, -2)
]

# Create the adjacency list for the 6x6 board
def get_knight_moves(x, y):
    moves = []
    for dx, dy in knight_moves:
        new_x = x + dx
        new_y = y + dy
        if 1 <= new_x <= 6 and 1 <= new_y <= 6:
            moves.append((new_x, new_y))
    return moves

# Create the board representation
board = {}
for row in range(1, 7):
    for col in range(1, 7):
        board[(row, col)] = get_knight_moves(row, col)

# Print the adjacency list

# Map 
l = [1, 2, 3, 4, 5, 6]
rows = {i: 7 - i for i in l[::-1]}
cols = {6-i: k  for i, k in enumerate(['a','b','c','d','e','f'][::-1])}
f = lambda x: cols[x[0]] + str(rows[x[1]])

remapped_board = dict()
for position, moves in board.items():
    remapped_position = f(position)
    remapped_moves = list(map(f, moves))
    remapped_board[remapped_position] = remapped_moves                          
    
print(remapped_board)

{'a6': ['c5', 'b4'], 'a5': ['c4', 'c6', 'b3'], 'a4': ['c3', 'c5', 'b2', 'b6'], 'a3': ['c2', 'c4', 'b1', 'b5'], 'a2': ['c1', 'c3', 'b4'], 'a1': ['c2', 'b3'], 'b6': ['d5', 'c4', 'a4'], 'b5': ['d4', 'd6', 'c3', 'a3'], 'b4': ['d3', 'd5', 'c2', 'c6', 'a2', 'a6'], 'b3': ['d2', 'd4', 'c1', 'c5', 'a1', 'a5'], 'b2': ['d1', 'd3', 'c4', 'a4'], 'b1': ['d2', 'c3', 'a3'], 'c6': ['e5', 'a5', 'd4', 'b4'], 'c5': ['e4', 'e6', 'a4', 'a6', 'd3', 'b3'], 'c4': ['e3', 'e5', 'a3', 'a5', 'd2', 'd6', 'b2', 'b6'], 'c3': ['e2', 'e4', 'a2', 'a4', 'd1', 'd5', 'b1', 'b5'], 'c2': ['e1', 'e3', 'a1', 'a3', 'd4', 'b4'], 'c1': ['e2', 'a2', 'd3', 'b3'], 'd6': ['f5', 'b5', 'e4', 'c4'], 'd5': ['f4', 'f6', 'b4', 'b6', 'e3', 'c3'], 'd4': ['f3', 'f5', 'b3', 'b5', 'e2', 'e6', 'c2', 'c6'], 'd3': ['f2', 'f4', 'b2', 'b4', 'e1', 'e5', 'c1', 'c5'], 'd2': ['f1', 'f3', 'b1', 'b3', 'e4', 'c4'], 'd1': ['f2', 'b2', 'e3', 'c3'], 'e6': ['c5', 'f4', 'd4'], 'e5': ['c4', 'c6', 'f3', 'd3'], 'e4': ['c3', 'c5', 'f2', 'f6', 'd2', 'd6'], 'e3': ['c

# Util Mapping

In [4]:
group_to_value = {
    'a': 1,
    'b': 2,
    'c': 253
}

group_to_node = {
    'a': ['a6','a5','a4','a3','a2','a1',
          'b4','b3','b2','b1',
          'c2','c1'
         ],
    'b': ['b6','b5',
          'c6','c5','c4','c3',
          'd4','d3','d2','d1',
          'e2','e1'
         ],
    'c': ['d6','d5',
          'e6','e5','e4','e3',
          'f6','f5','f4','f3','f2','f1'
         ]
}

node_to_group = dict()

for k, v in group_to_node.items():
    for node in v:
        node_to_group[node] = k
        
        
        

# Search

In [5]:
g = nx.Graph(remapped_board)

In [6]:
from simanneal import Annealer
import random
from collections import Counter

# a + b + c <= 50
# MIN: 1 + 2 + 3 = 6
# MAX: 17+15+18 = 50

# How triplets with distict integers added up to fifty or less?

group_to_value = {
    'a': 11,
    'b': 5,
    'c': 2
}

group_to_node = {
    'a': ['a6','a5','a4','a3','a2','a1',
          'b4','b3','b2','b1',
          'c2','c1'
         ],
    'b': ['b6','b5',
          'c6','c5','c4','c3',
          'd4','d3','d2','d1',
          'e2','e1'
         ],
    'c': ['d6','d5',
          'e6','e5','e4','e3',
          'f6','f5','f4','f3','f2','f1'
         ]
}

node_to_group = dict()

for k, v in group_to_node.items():
    for node in v:
        node_to_group[node] = k
        
        


class KnightsMoveProblem(Annealer):
    def __init__(self, state, rewards):
        super(KnightsMoveProblem, self).__init__(state)  # important!
        self.G = nx.Graph(remapped_board)
        self.rewards = rewards
    
    def add_node(self):
        '''
        It is not possible to add just one node to a path.

        It has to be done in pairs due to the knights move condition
        '''
        G = self.G
        path = self.state
        initial_energy = self.energy()
        # Select two nodes that are 1 index apart
        n = len(path)
        choice = np.random.randint(0, n-1)
        start = path[choice]
        end = path[choice+1]
        G.remove_edge(start, end)
        sub_paths = nx.all_shortest_paths(G, start, end)
        G.add_edge(start, end)
        beginning = path[:choice]
        ending = path[choice+2:]
        for sp in sub_paths:
            res = all(map(lambda x: (x not in beginning) & (x not in ending), sp))
            new_path = beginning + sp + ending
            assert (sum(Counter(path).values()) == len(Counter(path).keys()))
            if (res) and (len(new_path) >= n):
                self.state = new_path
                return self.energy() - initial_energy

        
        return int(self.energy() - initial_energy)

    
    def remove_node(self):
        G = self.G
        path = self.state
        initial_energy = self.energy()
        n = len(path)
        if n == 5:
            self.state = path
            return int(self.energy() - initial_energy)
        
        # Select two nodes that are 3 indicies apart
        choices = [i for i in range(0, n-3)]
        for i in range(100):
            choice = np.random.choice(choices, 1)[0]
            start = path[choice]
            end = path[choice+3]
            sub_paths = nx.all_shortest_paths(G, start, end)
            # Find the shortest path
            for sp in sub_paths:
                res = all(map(lambda x: (x not in path[:choice]) & (x not in path[choice+3+1:]), sp))
                assert (sum(Counter(path).values()) == len(Counter(path).keys()))
                new_path = path[:choice] + sp + path[choice+3+1:] 
                if (res) & (len(new_path) <= len(path)):
                    self.state = new_path
                    return int(self.energy() - initial_energy)
            choices.remove(choice)
            
        # self.state = new_path
        # print(self.energy() - initial_energy)
        return int(self.energy() - initial_energy)
        
    def move(self):
        """
        Make the state changes

        1. Add Nodes
        2. Remove Nodes
        """
        initial_energy = self.energy()
        direction = random.random()
        if direction > 0.5:
            e = self.add_node()
            return e
        else:
            e = self.remove_node() 
            return e

    def eval_path(self, path):
        score = 0
        previous_node = None
        for node in path:
            if previous_node:
                previous_group = node_to_group[previous_node]
                current_group = node_to_group[node]
                prev_score = score
                score = score * self.rewards[node_to_group[node]] if current_group != previous_group else score + self.rewards[node_to_group[node]]
                previous_node = node
            else:
                prev_score = score
                score += self.rewards[node_to_group[node]]
                previous_node = node
        return score
    
    def energy(self):
        score = self.eval_path(self.state)
        energy = np.abs(score - 2024)
        return int(energy)

# Optimize Lengths

In [7]:
# You can adjust the key map to find a solution
# or write some code so try different permutations of factors
key_map = {'a': 1, 'b': 3, 'c': 2}

min_trip_len = 100
trip_a = []
e = 10000000

for i in range(10):
    seed_path = nx.all_simple_paths(g, 'a1', 'f6')
    tsp = KnightsMoveProblem(next(seed_path), key_map)
    t_a, t_e = tsp.anneal()
    if (t_e == 0) and (len(t_a) < min_trip_len):
        trip_a = t_a
        e = t_e   
        min_trip_len = len(t_a)
        break

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         10.00    72.40%     0.40%     0:00:03     0:00:00

In [8]:
min_trip_len = 100
trip_b = []
e = 10000000

for i in range(10):
    seed_path = nx.all_simple_paths(g, 'a6', 'f1')
    tsp = KnightsMoveProblem(next(seed_path), key_map)
    t_b, t_e = tsp.anneal()
    if (t_e == 0) and (len(t_b) < min_trip_len):
        trip_b = t_b
        e = t_e   
        min_trip_len = len(t_b)
        break

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          2.00    46.20%     0.00%     0:00:03     0:00:00

In [9]:
print(len(trip_a))
print(len(trip_b))

21
25


In [10]:
print(trip_a)
print(trip_b)

['a1', 'b3', 'd4', 'c6', 'e5', 'c4', 'd6', 'e4', 'c5', 'd3', 'b4', 'a2', 'c1', 'e2', 'c3', 'b1', 'a3', 'c2', 'e3', 'd5', 'f6']
['a6', 'c5', 'b3', 'a1', 'c2', 'e1', 'd3', 'b4', 'c6', 'e5', 'f3', 'd2', 'c4', 'a3', 'b1', 'c3', 'e2', 'f4', 'd5', 'f6', 'e4', 'd6', 'f5', 'e3', 'f1']


In [11]:
def eval_path(path, val_map):
    score = 0
    previous_node = None
    for node in path:
        if previous_node:
            previous_group = node_to_group[previous_node]
            current_group = node_to_group[node]
            prev_score = score
            score = score * val_map[node_to_group[node]] if current_group != previous_group else score + val_map[node_to_group[node]]
            print(f'{previous_node} -> {node} {prev_score} -> {score} {previous_group} -> {current_group}')
            previous_node = node
        else:
            prev_score = score
            
            score += val_map[node_to_group[node]]
            print(f'{previous_node} -> {node} {prev_score} -> {score}')
            previous_node = node
            
    return score, 

In [12]:
key_map = {'a': 1, 'b': 3, 'c': 2}
trip_a = ['a1', 'b3', 'a5', 'c4', 'e3', 'd1', 'f2', 'e4', 'c5', 'd3', 'b2', 'a4', 'c3', 'e2', 'd4', 'c6', 'b4', 'd5', 'f6']
trip_b = ['a6', 'b4', 'a2', 'c3', 'b5', 'a3', 'c4', 'a5', 'b3', 'c5', 'a4', 'b2', 'd3', 'c1', 'e2', 'd4', 'c2', 'e3', 'f1']

eval_path(trip_a, key_map)

None -> a1 0 -> 1
a1 -> b3 1 -> 2 a -> a
b3 -> a5 2 -> 3 a -> a
a5 -> c4 3 -> 9 a -> b
c4 -> e3 9 -> 18 b -> c
e3 -> d1 18 -> 54 c -> b
d1 -> f2 54 -> 108 b -> c
f2 -> e4 108 -> 110 c -> c
e4 -> c5 110 -> 330 c -> b
c5 -> d3 330 -> 333 b -> b
d3 -> b2 333 -> 333 b -> a
b2 -> a4 333 -> 334 a -> a
a4 -> c3 334 -> 1002 a -> b
c3 -> e2 1002 -> 1005 b -> b
e2 -> d4 1005 -> 1008 b -> b
d4 -> c6 1008 -> 1011 b -> b
c6 -> b4 1011 -> 1011 b -> a
b4 -> d5 1011 -> 2022 a -> c
d5 -> f6 2022 -> 2024 c -> c


(2024,)

In [13]:
eval_path(trip_b, key_map)


None -> a6 0 -> 1
a6 -> b4 1 -> 2 a -> a
b4 -> a2 2 -> 3 a -> a
a2 -> c3 3 -> 9 a -> b
c3 -> b5 9 -> 12 b -> b
b5 -> a3 12 -> 12 b -> a
a3 -> c4 12 -> 36 a -> b
c4 -> a5 36 -> 36 b -> a
a5 -> b3 36 -> 37 a -> a
b3 -> c5 37 -> 111 a -> b
c5 -> a4 111 -> 111 b -> a
a4 -> b2 111 -> 112 a -> a
b2 -> d3 112 -> 336 a -> b
d3 -> c1 336 -> 336 b -> a
c1 -> e2 336 -> 1008 a -> b
e2 -> d4 1008 -> 1011 b -> b
d4 -> c2 1011 -> 1011 b -> a
c2 -> e3 1011 -> 2022 a -> c
e3 -> f1 2022 -> 2024 c -> c


(2024,)

# Need to check Path Validity




In [14]:
from collections import Counter

def accepts(g, path, trip_type='a'):
    if trip_type == 'a':
        cond = (path[0] == 'a1') & (path[-1] == 'f6') & (sum(Counter(path).values()) == len(Counter(path).keys()))
    else:
        cond = (path[0] == 'a6') & (path[-1] == 'f1') & (sum(Counter(path).values()) == len(Counter(path).keys()))
    return cond & all([(path[i], path[i+1]) in g.edges() for i in range(len(path)-1)])

def unique(trip):
    return len(set(trip)) == len(trip)

In [15]:
accepts(nx.Graph(remapped_board), trip_a, trip_type='a')

True

In [16]:
unique(trip_a[1])

True

In [17]:
accepts(nx.Graph(remapped_board), trip_b, trip_type='b')

True

In [18]:
unique(trip_b)

True

# Results

In [19]:
# VAL MAP ({'a': 42, 'b': 6, 'c': 1},)
# Trip A ['a1', 'b3', 'c1', 'a2', 'b4', 'c2', 'a3', 'b1', 'c3', 'b5', 'd6', 'e4', 'f6']
# Trip B ['a6', 'b4', 'a2', 'c1', 'b3', 'a1', 'c2', 'a3', 'b5', 'c3', 'd5', 'e3', 'f1']

In [20]:
# These are the optimal paths that I found and submitted
# key_map = {'a': 1, 'b': 3, 'c': 2}
# trip_a = ['a1', 'b3', 'a5', 'c4', 'e3', 'd1', 'f2', 'e4', 'c5', 'd3', 'b2', 'a4', 'c3', 'e2', 'd4', 'c6', 'b4', 'd5', 'f6']
# trip_b = ['a6', 'b4', 'a2', 'c3', 'b5', 'a3', 'c4', 'a5', 'b3', 'c5', 'a4', 'b2', 'd3', 'c1', 'e2', 'd4', 'c2', 'e3', 'f1']

def print_result(km, ta, tb):
    km_ = f'{key_map["a"]},{key_map["b"]},{key_map["c"]}'
    trip_a_ = ','.join(trip_a)
    trip_b_ = ','.join(trip_b)
    return f"{km_},{trip_a_},{trip_b_}"

In [21]:
print_result(key_map, trip_a, trip_b)

'1,3,2,a1,b3,a5,c4,e3,d1,f2,e4,c5,d3,b2,a4,c3,e2,d4,c6,b4,d5,f6,a6,b4,a2,c3,b5,a3,c4,a5,b3,c5,a4,b2,d3,c1,e2,d4,c2,e3,f1'